# 3d mitochondria visualizer using vtk
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


In [6]:
astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
astrocytes.head()

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,88240,60656,256,648518346349517321,glia,astrocyte
1,72368,56912,590,648518346341392909,glia,astrocyte
4,72704,54688,506,648518346349536799,glia,astrocyte
5,74576,75520,1163,648518346349525537,glia,astrocyte
6,79328,67360,445,648518346349525544,glia,astrocyte


In [7]:
astrocyte_list = astrocytes.cell_segid.to_list()
print(astrocyte_list)

[648518346349517321, 648518346341392909, 648518346349536799, 648518346349525537, 648518346349525544, 648518346349526583, 648518346349490239, 648518346349524070, 648518346349386860, 648518346349508722, 648518346349536888, 648518346349516953, 648518346349521062, 648518346342468778, 648518346342795947, 648518346349524139, 648518346342917290, 648518346349528249, 648518346349528250, 648518346349507266, 648518346349498574, 648518346349528271, 648518346349525715, 648518346341399264, 648518346349536487, 648518346349532924, 648518346349527316, 648518346349527317, 648518346349527319, 648518346342807831, 648518346349537050, 648518346349538089, 648518346341356348, 648518346349517141, 648518346349539158, 648518346349525862, 648518346344075119, 648518346341354380, 648518346349528465, 648518346349484948, 648518346342797245, 648518346349521344, 648518346342795202, 648518346349530569]


# Use index value to view an astrocyte from list 

In [29]:
# pull all mitos from a cellid of interest from astrocyte list, by index number

indx = 12
cellid = astrocyte_list[indx]

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf.sort_values(by='mito_vx', ascending=False)


,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
93186,3725121,14569500,111174,50256,2053,108102,49238,1755,113764,51658,2157,648518346349521062,398002.92,179916.48,82120
4872,3717670,10137260,112176,48446,1949,109536,46138,1756,113452,49960,2138,648518346349521062,401590.08,173436.68,77960
1343,3734759,5755800,109724,55454,1891,108510,53760,1731,110738,56528,1996,648518346349521062,392811.92,198525.32,75640
2397247,4071245,5281152,115310,52806,2081,114380,51214,2027,116022,53754,2157,648518346349521062,412809.80,189045.48,83240
176243,3837233,3592932,111128,50438,2053,110154,49922,2001,112532,51674,2088,648518346349521062,397838.24,180568.04,82120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81642,3837564,1236,110482,50824,2124,110470,50810,2123,110498,50844,2126,648518346349521062,395525.56,181949.92,84960
80532,3942873,1220,113722,46842,2000,113710,46830,1999,113738,46856,2002,648518346349521062,407124.76,167694.36,80000
80044,3954686,1212,112392,52088,2136,112374,52070,2136,112410,52108,2138,648518346349521062,402363.36,186475.04,85440
80183,4189985,1212,116738,54878,1903,116722,54864,1901,116752,54892,1906,648518346349521062,417922.04,196463.24,76120


In [30]:
cellid

648518346349521062

In [31]:
thresh = 2000000

In [32]:
mito_query_abovethresh = mito_querydf[mito_querydf.mito_vx > thresh]
mito_query_abovethresh

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
367,3613492,2601224,108604,51468,1582,107598,50950,1505,109032,51808,1651,648518346349521062,388802.32,184255.44,63280
1343,3734759,5755800,109724,55454,1891,108510,53760,1731,110738,56528,1996,648518346349521062,392811.92,198525.32,75640
4872,3717670,10137260,112176,48446,1949,109536,46138,1756,113452,49960,2138,648518346349521062,401590.08,173436.68,77960
7539,3947322,2207736,114932,49658,1741,113752,48966,1663,115982,50276,1824,648518346349521062,411456.56,177775.64,69640
93186,3725121,14569500,111174,50256,2053,108102,49238,1755,113764,51658,2157,648518346349521062,398002.92,179916.48,82120
176243,3837233,3592932,111128,50438,2053,110154,49922,2001,112532,51674,2088,648518346349521062,397838.24,180568.04,82120
1399434,3617672,2223360,108012,52970,1890,107434,52724,1861,109466,53250,1928,648518346349521062,386682.96,189632.60,75600
2361502,3946354,3579648,113134,48946,2080,112082,47250,2022,113940,49942,2157,648518346349521062,405019.72,175226.68,83200
2397247,4071245,5281152,115310,52806,2081,114380,51214,2027,116022,53754,2157,648518346349521062,412809.80,189045.48,83240


In [33]:
mito_query_belowthresh = mito_querydf[mito_querydf.mito_vx <= thresh]
mito_query_belowthresh

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
1541,3942375,718024,113364,46750,1952,113122,46342,1938,113650,47120,1991,648518346349521062,405843.12,167365.00,78080
4863,3951807,45420,113968,49756,2154,113902,49692,2150,114040,49858,2157,648518346349521062,408005.44,178126.48,86160
5620,4068399,52116,116118,50446,1690,116060,50412,1684,116204,50486,1698,648518346349521062,415702.44,180596.68,67600
6802,3722373,71728,109886,47862,2053,109762,47770,2042,109948,47968,2067,648518346349521062,393391.88,171345.96,82120
6806,3608579,10720,106872,49022,1850,106800,48988,1847,106974,49050,1857,648518346349521062,382601.76,175498.76,74000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409276,3724908,1427580,108138,49776,1599,107888,49278,1570,108586,50082,1652,648518346349521062,387134.04,178198.08,63960
2409329,3729993,1480828,108852,51512,1690,108600,51088,1612,109182,51846,1795,648518346349521062,389690.16,184412.96,67600
2409337,3951244,1490440,113500,49760,1712,113190,49048,1608,113828,50228,1798,648518346349521062,406330.00,178140.80,68480
2409503,3833086,1833524,110824,48408,1744,110180,48142,1670,111404,48706,1805,648518346349521062,396749.92,173300.64,69760


In [34]:
mito_query_abovethresh_mitolist = mito_query_abovethresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_abovethresh_mitolist)))
print(f"type: "+str(type(mito_query_abovethresh_mitolist)))
print('')
# print(mito_query_abovethresh_mitolist) # uncomment to print the mito id list

length: 9
type: <class 'list'>



In [35]:
mito_query_belowthresh_mitolist = mito_query_belowthresh.mito_id.to_list()
print(f"length: "+str(len(mito_query_belowthresh_mitolist)))
print(f"type: "+str(type(mito_query_belowthresh_mitolist)))
print('')
# print(mito_query_belowthresh_mitolist) # uncomment to print the mito id list

length: 839
type: <class 'list'>



# vtk 3d viewer

In [36]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://seungdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [37]:
cell_id = cellid

In [38]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [39]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [40]:
len(mito_query_abovethresh_mitolist)

9

In [41]:
len(mito_query_belowthresh_mitolist)

839

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [ ]:
# download the mito meshes for this cell

for i in range(len(mito_query_abovethresh_mitolist)):
    mito_id = mito_query_abovethresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_id = mito_query_belowthresh_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)


In [ ]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    mito_seg_id = mito_query_abovethresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_abovethresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_abovethresh)


var_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    mito_seg_id = mito_query_belowthresh_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_belowthresh['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator_belowthresh)


In [ ]:
len(var_iterator_abovethresh)

In [ ]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_abovethresh = 1 # 0.35
mito_color_abovethresh = (0.2, 0.4, 0.7)
mito_opac_belowthresh = 1
mito_color_belowthresh = (0.7, 0.2, 0.2)

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator_abovethresh = {}
for i in range(len(mito_query_abovethresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_abovethresh['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator_abovethresh['mito_mesh_'+str(i)], opacity=mito_opac_abovethresh, color=(mito_color_abovethresh))
    
locals().update(actor_iterator_abovethresh)

actor_iterator_belowthresh = {}
for i in range(len(mito_query_belowthresh_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_belowthresh['mito_mesh_actor_' + str(i+(len(var_iterator_abovethresh)))] = trimesh_vtk.mesh_actor(var_iterator_belowthresh['mito_mesh_'+str(i)], opacity=mito_opac_belowthresh, color=(mito_color_belowthresh))
    
locals().update(actor_iterator_belowthresh)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_abovethresh)
actor_iterator.update(actor_iterator_belowthresh)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


In [ ]:
# save image to disk

camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

trimesh_vtk.render_actors(actor_iterator.values(),
                          filename='vtk_images/'+str(cellid)+'.png',
                          do_save=True,
                          scale=6,
                          camera=camera
                         )

In [26]:
cellid

648518346349530569

In [27]:
filename

NameError: name 'filename' is not defined

In [ ]:
# 648518346349536888 has two to three astrocytes

In [ ]:
# 648518346341354380 very interesting case

In [28]:
astrocyte_list

[648518346349517321,
 648518346341392909,
 648518346349536799,
 648518346349525537,
 648518346349525544,
 648518346349526583,
 648518346349490239,
 648518346349524070,
 648518346349386860,
 648518346349508722,
 648518346349536888,
 648518346349516953,
 648518346349521062,
 648518346342468778,
 648518346342795947,
 648518346349524139,
 648518346342917290,
 648518346349528249,
 648518346349528250,
 648518346349507266,
 648518346349498574,
 648518346349528271,
 648518346349525715,
 648518346341399264,
 648518346349536487,
 648518346349532924,
 648518346349527316,
 648518346349527317,
 648518346349527319,
 648518346342807831,
 648518346349537050,
 648518346349538089,
 648518346341356348,
 648518346349517141,
 648518346349539158,
 648518346349525862,
 648518346344075119,
 648518346341354380,
 648518346349528465,
 648518346349484948,
 648518346342797245,
 648518346349521344,
 648518346342795202,
 648518346349530569]